In [6]:
from datasets import load_dataset, load_metric



In [7]:
metric = load_metric('sacrebleu')

C:\Users\danie\AppData\Local\Temp\ipykernel_10232\1900502390.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('sacrebleu')


In [3]:
metric

Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions (`list` of `str`): list of translations to score. Each translation should be tokenized into a list of tokens.
    references (`list` of `list` of `str`): A list of lists of references. The contents of the first sub-list are the references for the first prediction, the contents of the second sub-list are for the second prediction, etc. Note that there must be the same number of references for each prediction (i.e. all sub-lists must be of the same length).
    smooth_method (`str`): The smoothing method to use, defaults to `'exp'`. Possible values are:
        - `'none'`: no smoothing
        - `'floor'`: increment zero counts
        - `'add-k'`: increment num/deno

In [4]:
predictions = ['''
Strammingen på kjedet bør settes slik at du kan trekke den ca 1 unna oppreist når vognen er ca 5 fot fra bakken.''']
references = [['''
Spenningen på kjedet bør settes slik at du kan trekke den ca 1 bort fra oppreist når vognen er rundt 5 meter fra bakken.''']]
sacrebleu = metric
results = sacrebleu.compute(predictions=predictions, references=references)
print(round(results["score"],1))



61.0


In [5]:
with open('sentence_pair_test.jsonl', 'r', encoding='utf-8') as f:
    dataset = f.readlines()

dataset_list = [eval(i)['translation'] for i in dataset]
dataset_list

import pandas as pd

df = pd.DataFrame.from_dict(dataset_list)
df.head()

,en,no
0,This is because the right side is displaced fo...,Dette skyldes at høyre side forskyves fremover...
1,Add headset spacers under the upper plate as n...,Legg styrelager spacere under den øvre platen ...
2,Repeat impacting the BBT-30.4 driver until one...,Fortsett å bruk BBT-30.4 til en lagerskålene e...
3,Impacts to the side of the rim from falling do...,Påvirkning på siden av felgen fra fall eller k...
4,"Check the dish reference on both sides, and re...",Kontroller sentreringsreferansen på begge side...


In [6]:
from transformers import MarianMTModel, MarianTokenizer


def translator_initial(src_text):
    input_text = f'>>nob<<{src_text}'
    model_name_initial = 'Helsinki-NLP/opus-mt-tc-big-en-gmq'
    tokenizer_initial = MarianTokenizer.from_pretrained(model_name_initial)
    model_initial = MarianMTModel.from_pretrained(model_name_initial)
    translated_initial = model_initial.generate(**tokenizer_initial(input_text, return_tensors="pt", padding=True))
    result_initial = [tokenizer_initial.decode(t, skip_special_tokens=True) for t in translated_initial]

    print(result_initial)

    return result_initial[0]

def translator_finetuned(src_text):
    input_text = f'>>nob<<{src_text}'
    model_name = 'opus-mt-tc-big-en-gmq-finetuned-en-to-no/checkpoint-4000'
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    translated = model.generate(**tokenizer(input_text, return_tensors="pt", padding=True))
    result = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

    print(result)

    return result[0]

In [7]:
df['initial_translation'] = df['en'].apply(lambda x: translator_initial(x) )

C:\Users\danie\PycharmProjects\Bicycle_translator\bike_env\lib\site-packages\transformers\generation\utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['Dette skyldes at den høyre siden er forskjøvet fremover av hengerbraketten.']
['Legg hodesettet avstandsstykker under den øvre platen etter behov.']
['Gjenta å påvirke BBT-30.4 driveren til en lagerkopp er helt ut.']
['Virkninger på siden av felgen fra å falle ned eller krasje kan også forårsake skade.']
['Kontroller tallerkenreferansen på begge sider, og gjenta etter behov til hjulet viser en 1 mm eller mindre gap ved navet.']
['Det anbefales også å bruke beskyttelseshansker som MG-2, og bruke øyebeskyttelse.']
['Det er en vane for mange mekanikere å sette trådlåseforbindelsen på den eksterne tråden.']
['Utformingen er en variant av cantileverbremsen.']
['Plukk opp forsiden av sykkelen, og forsiktig svinge styret frem og tilbake fra midten.']
['Avid skivekaliperbremser bruker et kule-og-socket-system for kalibermonteringsboltene.']
['Det høyre bildet er den ekstreme innoverbevegelsen til klossen, eller mer andefot.']
['Bruk en kjederenser som Park Tool CM-5.3 Cyclone Chain Scrubber.

In [9]:
df.head()

,en,no,initial_translation
0,This is because the right side is displaced fo...,Dette skyldes at høyre side forskyves fremover...,Dette skyldes at den høyre siden er forskjøvet...
1,Add headset spacers under the upper plate as n...,Legg styrelager spacere under den øvre platen ...,Legg hodesettet avstandsstykker under den øvre...
2,Repeat impacting the BBT-30.4 driver until one...,Fortsett å bruk BBT-30.4 til en lagerskålene e...,Gjenta å påvirke BBT-30.4 driveren til en lage...
3,Impacts to the side of the rim from falling do...,Påvirkning på siden av felgen fra fall eller k...,Virkninger på siden av felgen fra å falle ned ...
4,"Check the dish reference on both sides, and re...",Kontroller sentreringsreferansen på begge side...,"Kontroller tallerkenreferansen på begge sider,..."


In [10]:
df['finetuned_translation'] = df['en'].apply(lambda x:  translator_finetuned(x) )

C:\Users\danie\PycharmProjects\Bicycle_translator\bike_env\lib\site-packages\transformers\generation\utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['Dette skyldes at den høyre siden er forskjøvet fremover av girøret.']
['Legg styrelager spacere under den øvre platen om nødvendig.']
['Gjennsett slagene på BBT-30,4 driveren til en lagerskål er helt tom.']
['Virkninger på siden av felgen fra å falle ned eller krasje kan også forårsake skade.']
['Sjekk sentreringsreferansen på begge sider, og gjenta om nødvendig til hjulet viser et gap på 1 mm eller mindre ved navet.']
['Det anbefales også å bruke beskyttelseshansker som MG-2, og bruke øyebeskyttelse.']
['Det er en vane for mange mekanikere å sette gjengelåsingen på den utvendige gjengen.']
['Konstruksjonen er en variant av cantilever brems.']
['Plukk opp forsiden av sykkelen, og forsiktig sving styret frem og tilbake fra sentrum.']
['Avid skivebremser bruker et kulelager system for monteringsbolter.']
['Høyre bilde er den ekstreme innovergående bevegelse av cleats, eller mer duck fot.']
['Bruk en kjede rengjøringsmiddel som Park Tool CM-5.3 Cyclone Chain Scrubber.']
['Det neste måle

In [11]:
df.head()

,en,no,initial_translation,finetuned_translation
0,This is because the right side is displaced fo...,Dette skyldes at høyre side forskyves fremover...,Dette skyldes at den høyre siden er forskjøvet...,Dette skyldes at den høyre siden er forskjøvet...
1,Add headset spacers under the upper plate as n...,Legg styrelager spacere under den øvre platen ...,Legg hodesettet avstandsstykker under den øvre...,Legg styrelager spacere under den øvre platen ...
2,Repeat impacting the BBT-30.4 driver until one...,Fortsett å bruk BBT-30.4 til en lagerskålene e...,Gjenta å påvirke BBT-30.4 driveren til en lage...,"Gjennsett slagene på BBT-30,4 driveren til en ..."
3,Impacts to the side of the rim from falling do...,Påvirkning på siden av felgen fra fall eller k...,Virkninger på siden av felgen fra å falle ned ...,Virkninger på siden av felgen fra å falle ned ...
4,"Check the dish reference on both sides, and re...",Kontroller sentreringsreferansen på begge side...,"Kontroller tallerkenreferansen på begge sider,...","Sjekk sentreringsreferansen på begge sider, og..."


In [8]:

def compute_blue(pred,ref):

    sacrebleu = metric

    results = sacrebleu.compute(predictions=[[pred]], references=[[ref]])

    print(round(results["score"],1))
    return round(results["score"],1)



In [13]:
df['initial_translation_bleu'] = df[['initial_translation','no']].apply(lambda x: compute_blue(*x), axis=1)

8.2
47.0
8.5
38.8
52.8
72.2
14.4
27.8
50.9
8.1
43.9
67.7
49.2
31.6
52.9
64.4
84.3
52.8
43.4
26.6
46.5
21.5
52.0
61.7
61.7
12.5
31.5
22.2
44.8
15.9
55.0
12.4
22.4
36.7
71.3
34.2
44.6
36.7
53.1
56.3
15.4
28.0
26.2
60.7
11.7
56.8
12.5
13.1
57.1
39.4
69.1
61.7
17.0
66.2
62.6
74.1
10.8
23.5
62.4
44.5
4.1
46.2
48.6
80.4
4.6
10.1
47.5
39.3
26.6
53.0
41.9
34.9
53.1
43.3
48.8
42.3
25.6
61.3
3.5
64.7
57.1
44.5
63.8
40.1
23.6
48.4
42.6
74.0
64.8
49.3
46.2
20.6
7.3
67.7
44.5
30.3
43.4
39.7
56.6
55.5


In [14]:
df.head()

,en,no,initial_translation,finetuned_translation,initial_translation_bleu
0,This is because the right side is displaced fo...,Dette skyldes at høyre side forskyves fremover...,Dette skyldes at den høyre siden er forskjøvet...,Dette skyldes at den høyre siden er forskjøvet...,8.2
1,Add headset spacers under the upper plate as n...,Legg styrelager spacere under den øvre platen ...,Legg hodesettet avstandsstykker under den øvre...,Legg styrelager spacere under den øvre platen ...,47.0
2,Repeat impacting the BBT-30.4 driver until one...,Fortsett å bruk BBT-30.4 til en lagerskålene e...,Gjenta å påvirke BBT-30.4 driveren til en lage...,"Gjennsett slagene på BBT-30,4 driveren til en ...",8.5
3,Impacts to the side of the rim from falling do...,Påvirkning på siden av felgen fra fall eller k...,Virkninger på siden av felgen fra å falle ned ...,Virkninger på siden av felgen fra å falle ned ...,38.8
4,"Check the dish reference on both sides, and re...",Kontroller sentreringsreferansen på begge side...,"Kontroller tallerkenreferansen på begge sider,...","Sjekk sentreringsreferansen på begge sider, og...",52.8


In [15]:
df['finetuned_translation_bleu'] = df[['finetuned_translation','no']].apply(lambda x: compute_blue(*x), axis=1)

8.2
39.6
7.9
38.8
57.2
72.2
14.9
57.1
77.0
10.1
20.1
49.0
49.2
58.3
29.8
56.1
53.5
66.5
61.3
63.7
71.7
17.9
52.0
61.7
77.0
43.5
70.1
53.1
56.0
33.3
62.2
22.0
8.2
44.9
78.1
21.8
44.6
27.2
72.2
52.9
26.7
12.9
30.1
66.0
45.6
46.9
11.2
13.7
36.7
59.7
42.0
41.2
40.3
85.4
50.2
70.1
29.9
23.5
74.0
39.9
4.1
17.5
47.9
69.0
26.1
26.8
57.6
16.8
44.5
24.6
49.1
39.6
72.2
69.1
30.4
46.6
51.3
61.3
11.6
75.6
57.1
24.1
79.4
36.8
23.6
55.5
39.4
56.4
64.8
59.0
69.1
20.6
7.3
26.6
47.4
61.3
43.4
59.7
70.1
59.7


In [16]:
df

,en,no,initial_translation,finetuned_translation,initial_translation_bleu,finetuned_translation_bleu
0,This is because the right side is displaced fo...,Dette skyldes at høyre side forskyves fremover...,Dette skyldes at den høyre siden er forskjøvet...,Dette skyldes at den høyre siden er forskjøvet...,8.2,8.2
1,Add headset spacers under the upper plate as n...,Legg styrelager spacere under den øvre platen ...,Legg hodesettet avstandsstykker under den øvre...,Legg styrelager spacere under den øvre platen ...,47.0,39.6
2,Repeat impacting the BBT-30.4 driver until one...,Fortsett å bruk BBT-30.4 til en lagerskålene e...,Gjenta å påvirke BBT-30.4 driveren til en lage...,"Gjennsett slagene på BBT-30,4 driveren til en ...",8.5,7.9
3,Impacts to the side of the rim from falling do...,Påvirkning på siden av felgen fra fall eller k...,Virkninger på siden av felgen fra å falle ned ...,Virkninger på siden av felgen fra å falle ned ...,38.8,38.8
4,"Check the dish reference on both sides, and re...",Kontroller sentreringsreferansen på begge side...,"Kontroller tallerkenreferansen på begge sider,...","Sjekk sentreringsreferansen på begge sider, og...",52.8,57.2
...,...,...,...,...,...,...
95,"To start, zero out the spoke nipples.","For å starte, null ut eike nipplene.","For å starte, null ut de talte brystvortene.","For å starte, null ut eike nipplene.",30.3,61.3
96,Insert the small end of the tool through the b...,Sett den lille enden av verktøyet gjennom kran...,Sett den lille enden av verktøyet gjennom bunn...,Sett den lille enden av verktøyet gjennom kran...,43.4,43.4
97,Pull chain snug to determine which link to cut.,Trekk kjedet tettsittende for å finne ut hvilk...,Trekk kjetting tettsittende for å finne ut hvi...,Trekk kjedet stramt for å finne ut hvilken len...,39.7,59.7
98,Put an adjustable wrench on the bridge and a w...,Sett en justerbar skiftenøkkel på broen og en ...,Sett en justerbar skiftenøkkel på broen og en ...,Sett en justerbar skiftenøkkel på broen og en ...,56.6,70.1


In [17]:
print(df['finetuned_translation_bleu'].mean())

45.099


In [18]:
print(df['initial_translation_bleu'].mean())

41.511


In [19]:
df.to_csv('evaluate_finetuned_translation.csv', encoding='latin1')

### Add manually collected google translate to the dataframe.

In [20]:
with open('sentence_pair_test_google.jsonl', 'r', encoding='utf-8') as f:
    dataset_google = f.readlines()

dataset_list_google = [eval(i)['translation'] for i in dataset_google]
dataset_list_google

import pandas as pd

df_google = pd.DataFrame.from_dict(dataset_list_google)
df_google.head()

,en,no
0,This is because the right side is displaced fo...,Dette er fordi høyre side er forskjøvet fremov...
1,Add headset spacers under the upper plate as n...,Legg til hodesettavstandsstykker under den øvr...
2,Repeat impacting the BBT-30.4 driver until one...,Gjenta støt på BBT-30.4-driveren til en lagers...
3,Impacts to the side of the rim from falling do...,Støt på siden av felgen fra fall ned eller kra...
4,"Check the dish reference on both sides, and re...","Sjekk tallerkenreferansen på begge sider, og g..."


In [21]:
df['google_translate'] = df_google['no'].apply(lambda x: x)

In [22]:
df['google_translation_bleu'] = df[['google_translate','no']].apply(lambda x: compute_blue(*x), axis=1)

8.5
47.0
18.9
47.6
38.8
31.5
12.9
27.8
29.7
7.9
28.4
67.7
49.2
36.7
46.6
40.0
63.4
40.1
26.0
24.7
60.9
19.5
52.0
44.1
77.0
34.6
45.8
33.9
24.2
8.5
46.5
12.0
22.4
36.7
54.5
18.4
35.1
20.0
49.2
39.7
37.6
17.8
22.6
55.6
36.2
36.4
11.1
6.4
57.1
24.9
81.3
33.7
15.5
76.0
50.4
53.3
10.8
23.5
62.4
50.9
4.1
10.6
48.6
69.0
8.2
2.7
47.5
39.3
13.4
9.9
51.4
61.9
53.1
15.7
25.5
32.4
41.3
61.3
4.4
50.5
57.1
7.9
79.4
33.9
34.5
52.0
27.7
74.0
23.5
41.6
48.5
20.6
7.3
67.7
33.3
36.7
43.4
17.6
56.6
55.5


In [23]:
print(df['google_translation_bleu'].mean())

36.615


In [1]:
import pandas as pd
df = pd.read_csv('evaluate_finetuned_translation.csv', encoding='latin1')
df.head()


,Unnamed: 0,en,no,initial_translation,finetuned_translation,initial_translation_bleu,finetuned_translation_bleu
0,0,This is because the right side is displaced fo...,Dette skyldes at høyre side forskyves fremover...,Dette skyldes at den høyre siden er forskjøvet...,Dette skyldes at den høyre siden er forskjøvet...,8.2,8.2
1,1,Add headset spacers under the upper plate as n...,Legg styrelager spacere under den øvre platen ...,Legg hodesettet avstandsstykker under den øvre...,Legg styrelager spacere under den øvre platen ...,47.0,39.6
2,2,Repeat impacting the BBT-30.4 driver until one...,Fortsett å bruk BBT-30.4 til en lagerskålene e...,Gjenta å påvirke BBT-30.4 driveren til en lage...,"Gjennsett slagene på BBT-30,4 driveren til en ...",8.5,7.9
3,3,Impacts to the side of the rim from falling do...,Påvirkning på siden av felgen fra fall eller k...,Virkninger på siden av felgen fra å falle ned ...,Virkninger på siden av felgen fra å falle ned ...,38.8,38.8
4,4,"Check the dish reference on both sides, and re...",Kontroller sentreringsreferansen på begge side...,"Kontroller tallerkenreferansen på begge sider,...","Sjekk sentreringsreferansen på begge sider, og...",52.8,57.2


In [3]:
with open('sentence_pair_test_google.jsonl', 'r', encoding='utf-8') as f:
    dataset_google = f.readlines()

dataset_list_google = [eval(i)['translation'] for i in dataset_google]
dataset_list_google


df_google = pd.DataFrame.from_dict(dataset_list_google)
df_google.head()

,en,no
0,This is because the right side is displaced fo...,Dette er fordi høyre side er forskjøvet fremov...
1,Add headset spacers under the upper plate as n...,Legg til hodesettavstandsstykker under den øvr...
2,Repeat impacting the BBT-30.4 driver until one...,Gjenta støt på BBT-30.4-driveren til en lagers...
3,Impacts to the side of the rim from falling do...,Støt på siden av felgen fra fall ned eller kra...
4,"Check the dish reference on both sides, and re...","Sjekk tallerkenreferansen på begge sider, og g..."


In [4]:
df['google_translate'] = df_google['no'].apply(lambda x: x)

In [5]:
df.head()


,Unnamed: 0,en,no,initial_translation,finetuned_translation,initial_translation_bleu,finetuned_translation_bleu,google_translate
0,0,This is because the right side is displaced fo...,Dette skyldes at høyre side forskyves fremover...,Dette skyldes at den høyre siden er forskjøvet...,Dette skyldes at den høyre siden er forskjøvet...,8.2,8.2,Dette er fordi høyre side er forskjøvet fremov...
1,1,Add headset spacers under the upper plate as n...,Legg styrelager spacere under den øvre platen ...,Legg hodesettet avstandsstykker under den øvre...,Legg styrelager spacere under den øvre platen ...,47.0,39.6,Legg til hodesettavstandsstykker under den øvr...
2,2,Repeat impacting the BBT-30.4 driver until one...,Fortsett å bruk BBT-30.4 til en lagerskålene e...,Gjenta å påvirke BBT-30.4 driveren til en lage...,"Gjennsett slagene på BBT-30,4 driveren til en ...",8.5,7.9,Gjenta støt på BBT-30.4-driveren til en lagers...
3,3,Impacts to the side of the rim from falling do...,Påvirkning på siden av felgen fra fall eller k...,Virkninger på siden av felgen fra å falle ned ...,Virkninger på siden av felgen fra å falle ned ...,38.8,38.8,Støt på siden av felgen fra fall ned eller kra...
4,4,"Check the dish reference on both sides, and re...",Kontroller sentreringsreferansen på begge side...,"Kontroller tallerkenreferansen på begge sider,...","Sjekk sentreringsreferansen på begge sider, og...",52.8,57.2,"Sjekk tallerkenreferansen på begge sider, og g..."


In [9]:
df['google_translation_bleu'] = df[['google_translate','no']].apply(lambda x: compute_blue(*x), axis=1)

8.5
47.0
18.9
47.6
38.8
31.5
12.9
27.8
29.7
7.9
28.4
67.7
49.2
36.7
46.6
40.0
63.4
40.1
26.0
24.7
60.9
19.5
52.0
44.1
77.0
34.6
45.8
33.9
24.2
8.5
46.5
12.0
22.4
36.7
54.5
18.4
35.1
20.0
49.2
39.7
37.6
17.8
22.6
55.6
36.2
36.4
11.1
6.4
57.1
24.9
81.3
33.7
15.5
76.0
50.4
53.3
10.8
23.5
62.4
50.9
4.1
10.6
48.6
69.0
8.2
2.7
47.5
39.3
13.4
9.9
51.4
61.9
53.1
15.7
25.5
32.4
41.3
61.3
4.4
50.5
57.1
7.9
79.4
33.9
34.5
52.0
27.7
74.0
23.5
41.6
48.5
20.6
7.3
67.7
33.3
36.7
43.4
17.6
56.6
55.5


In [17]:
df['finetuned_translation_bleu'] = df['finetuned_translation_bleu'].astype('str')

In [18]:
df.to_csv('evaluation_sacrebleu.csv', sep=';', encoding='latin1')